In [ ]:
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
pip install flair

In [ ]:
from flair.data import Sentence
from flair.nn import Classifier
from flair.models import SequenceTagger

In [ ]:
tagger = Classifier.load('ner-large')

In [ ]:
soup1_37 = BeautifulSoup(open("/Users/laurasoffiantini/Downloads/E265PlinyNaturalHistory.html", encoding="utf-8"), "html.parser")

In [ ]:
html1_37 = soup1_37.find_all('html')

In [ ]:
reference_column = []
index_column = []
token_column = []
start_pos_column = []
flair_ner_column = []

In [ ]:
for html in html1_37:
    
    paragraphs = html.find_all("p")
    
    for paragraph in paragraphs:
        
        p_tag_id = paragraph.get("id")
        print(p_tag_id)
        
        text = paragraph.get_text()
        sentence = Sentence(text)
        tagger.predict(sentence)
        
        reference_column_temp = []
        index_column_temp = []
        token_column_temp = []
        start_pos_column_temp = []
        flair_ner_column_temp = []
        
        for index, token in enumerate(sentence):
            
            token_text = token.text
            start_pos = token.start_position
            
            reference_column_temp.append(p_tag_id)
            index_column_temp.append(index)
            token_column_temp.append(token_text)
            start_pos_column_temp.append(start_pos)
            flair_ner_column_temp.append('O')
        
        for entity in sentence.get_spans('ner'):
            
            entity_label = entity.labels[0].value
            
            if str(entity_label) == 'LOC':
                
                for i, tok in enumerate(entity):
                    
                    start_pos_tok = tok.start_position
                    
                    if i == 0:
                        tok_label = 'B-'+str(entity_label)
                    else :
                        tok_label = 'I-'+str(entity_label)
                    
                    for x,y in enumerate(start_pos_column_temp):
                        
                        if int(y) == int(start_pos_tok):
                            flair_ner_column_temp[x] = tok_label
                            
        reference_column.extend(reference_column_temp)
        index_column.extend(index_column_temp)
        token_column.extend(token_column_temp)
        start_pos_column.extend(start_pos_column_temp)
        flair_ner_column.extend(flair_ner_column_temp)                 

In [ ]:
data = {'reference' : reference_column, 'index' : index_column, 'token' : token_column, 'start_pos' : start_pos_column, 'flair_ner' : flair_ner_column}

In [ ]:
flair_NH = pd.DataFrame(data)